In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import spacy
from spacy.lang.fr import French
from spacy.tokenizer import Tokenizer
from spacy.tokens import Doc
from spacy.vocab import Vocab
from spacy.matcher import Matcher
from spacy.matcher import PhraseMatcher
from textblob import TextBlob
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import re

/anaconda3/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [5]:
measurements = {"g": ["gr.", "gr", "g", "g.", "grams", "gramme"], "kg": ["kg.", "kg", "kilogramme"], "mg": ["mg.", "mg", "miligramme"],
                "ml": ["ml.", "ml", "milliliters", "mililitre"], "cl": ["cl.", "cl", "centilitre"], "l": ["l.", "l", "litre"],
                "cuillère à soupe": ["c. à soupe", "cuillère à soupe", "cs", "cas", "càs", "c. à table", "cuillère à table"],
                "cuillère à café": ["c. à café", "cuillère à café", "cc", "cac", "càc", "c. à thé", "cuillère à thé"],
                "tasse": ["tasse"], "bol": ["bol"], "verre": ["verre"], "filet": ["filet"], "zeste": ["zeste"], "pièce": ["pièce", "unité", "ampollas", "set"],
                "noisette": ["noisette"], "noix": ["noix"], "pincée": ["pincée"], "pointe":["pointe"], "poignée": ["poignée"],
                "feuille": ["feuille"], "branche": ["branche"], "gousse": ["gousse"], "tranche": ["tranche"], "cube": ["cube"],
                "boîte": ["boîte"], "barquette": ["barquette"], "pot": ["pot"], "bâtonnet": ["bâtonnet"], "boule": ["boule"],
                "rouleau": ["rouleau"], "p": ["p"]}

units = [item for sublist in measurements.values() for item in sublist]

quantities = { "½": 0.5, "1/2": 0.5, "1½": 1.5, "1 1/2": 0.5, "⅓": 0.33, "1/3": 0.33, "¼": 0.25, "1/5": 0.2, "¼": 0.25, "un demi": 0.5, "une demi": 0.5,"un et demi": 1.5,
              "tiers": 0.33, "quart": 0.25, "zero": 0, "deux": 2, "trois": 3, "quatre": 4,"cinq": 5, "six": 6, "sept": 7,
              "huit": 8, "neuf": 9, "dix": 10, "onze": 11, "douze": 12, "treize": 13, "quatorze": 14, "quinze": 15, "seize": 16,
              "dix-sept": 17, "dix-huit": 18, "dix-neuf": 19, "vingt": 20, "trente": 30, "quarante": 40, "cinquante": 50,
              "soixante": 60, "soixante-dix": 70, "quatre-vingt": 80, "quatre-vingt-dix": 90, "dizaine": 10, "une dizaine": 10,
              "douzaine": 12, "une douzaine": 12, "demi-douzaine": 6, "une demi-douzaine": 6, "vingtaine": 20, "une vingtaine": 20,
              "trentaine": 30, "quarantaine": 40, "cinquantaine": 50, "centaine": 100, "une centaine": 100, "cent": 100,
              "un": 1, "une": 1}

char_list = [i for i in "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZéèàâêïù▪️"]


In [16]:
web_product_data = pd.read_csv ('/Users/vincentsalamand/Downloads/products_systemeu.csv')
print(len(web_product_data))


8425


In [17]:
data = pd.DataFrame()
data['quantity_match'] = ""
data['unit_match'] = ""
data['price_match'] = ""
data['price_per_unit_match'] = ""
data = pd.concat([data, web_product_data], axis=1)
data['shelter'] = ""



In [37]:
data.head()

,quantity_match,unit_match,price_match,price_per_unit_match,shelter_main,shelter_parent,shelter_child,brand,description1,description,price,price_per_unit,store,image_url,url,ean,shelter
0,1.000,NaN,2.30,2.30,Fruits et Légumes,Fruits,Fruits bio,NaN,"banane cavendish, bio, catégorie 2, république...","Banane Cavendish, BIO, catégorie 2, République...",2.30,"2,30 € kg",Système U,https://www.coursesu.com/dw/image/v2/BBQX_PRD/...,/p/banane-cavendish-bio-categorie-2-republique...,2890030,
1,1.496,NaN,3.95,2.64,Fruits et Légumes,Fruits,Fruits bio,NaN,"orange salutiana, u bio, calibre 5/6, catégori...","Orange Salutiana, U BIO, calibre 5/6, catégori...",3.95,"2,64 € kg",Système U,https://www.coursesu.com/dw/image/v2/BBQX_PRD/...,/p/orange-salutiana-u-bio-calibre-5-6-categori...,5927091,
2,0.249,NaN,0.86,3.45,Fruits et Légumes,Fruits,Fruits bio,NaN,"orange naveline, bio, calibre 4, catégorie 2, ...","Orange naveline, BIO, calibre 4, catégorie 2, ...",0.86,"3,45 € kg",Système U,https://www.coursesu.com/dw/image/v2/BBQX_PRD/...,/p/orange-naveline-bio-calibre-4-categorie-2-e...,1396111,
3,1.000,NaN,3.95,3.95,Fruits et Légumes,Fruits,Fruits bio,NaN,"mandarine clemenvilla, u bio, calibre 1x-xx, c...","Mandarine Clemenvilla, U BIO, calibre 1X-XX, c...",3.95,"3,95 € kg",Système U,https://www.coursesu.com/dw/image/v2/BBQX_PRD/...,/p/mandarine-clemenvilla-u-bio-calibre-1x-xx-c...,5894870,
4,1.000,NaN,2.30,2.30,Fruits et Légumes,Fruits,"Bananes, Kiwis, Fruits exotiques",NaN,"banane cavendish, bio, catégorie 2, république...","Banane Cavendish, BIO, catégorie 2, République...",2.30,"2,30 € kg",Système U,https://www.coursesu.com/dw/image/v2/BBQX_PRD/...,/p/banane-cavendish-bio-categorie-2-republique...,2890030,


In [33]:
# clean prices
data['price_match'] = data.price
data['price_per_unit_match'] = data.price_per_unit.apply(lambda x: float(x.replace('.', '').replace(',', '.').split()[0]))
data.quantity_match = round(data.price_match / data.price_per_unit_match, 3)

data['price_per_unit'] = data.price_per_unit.apply(lambda x: x.replace('/', ' ').lower())


In [38]:
# match description to find unit
nlp = French()

def convert_unit(value):
    results = []
    result = [unit for unit, item in measurements.items() if value in item]
    if len(result) > 0:
        return result[0]

measurement_matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
measurement_patterns = list(nlp.tokenizer.pipe(list(units)))
measurement_matcher.add("MEASUREMENT_PATTERN", None, *measurement_patterns)
tokenizer = Tokenizer(nlp.vocab)

measurement_parser = []
price_per_unit = list(nlp.pipe(data.price_per_unit))

for line in price_per_unit:
    line = tokenizer(re.sub('[0-9]', ' ', line.text))
    matches = measurement_matcher(line)
    elements = []
    if len(matches) > 0:
        for match_id, start, end in matches:
            span = line[start:end]
            elements.append(span.text.lower())
        measurement_parser.append(convert_unit(elements[0]))
    else:
        text_blob_object = TextBlob(line.text)
        singular_line = ' '.join(text_blob_object.words.singularize())
        matches = measurement_matcher(tokenizer(singular_line))
        if len(matches) > 0:
            for match_id, start, end in matches:
                span = tokenizer(singular_line)[start:end]
                elements.append(span.text.lower())
            measurement_parser.append(convert_unit(elements[0]))
        else:
            plural_line = ' '.join(text_blob_object.words.pluralize())
            matches = measurement_matcher(tokenizer(plural_line))
            if len(matches) > 0:
                for match_id, start, end in matches:
                    span = tokenizer(plural_line)[start:end]
                    elements.append(span.text.lower())
                measurement_parser.append(convert_unit(elements[0]))
            else:
                 measurement_parser.append("")

data['unit_match'] = measurement_parser

In [39]:
# get shelter list

for index, product in data.iterrows():       
    shelter = []
    shelter.append(data.loc[index].shelter_parent)    
    shelter.append(data.loc[index].shelter_main)
    shelter.append(data.loc[index].shelter_child)    
    data.at[index,'shelter'] = shelter

In [44]:
data['url'] = data.url.apply(lambda x: 'https://www.coursesu.com' + x)

In [46]:
export_csv = data.to_csv (r'/Users/vincentsalamand/Downloads/systemeu_catalog.csv', index = None, header=True)


In [45]:
data.tail(500)

,quantity_match,unit_match,price_match,price_per_unit_match,shelter_main,shelter_parent,shelter_child,brand,description1,description,price,price_per_unit,store,image_url,url,ean,shelter
7925,0.749,l,3.85,5.14,"Vins, bières, alcools",Vins rouges,Loire,NaN,"vin rouge aoc anjou les hauts buis u, 75cl","Vin rouge AOC Anjou Les hauts buis U, 75cl",3.85,"5,14 € l",Système U,https://www.coursesu.com/dw/image/v2/BBQX_PRD/...,https://www.coursesu.com/p/vin-rouge-aoc-anjou...,4842386,"[Vins rouges, Vins, bières, alcools, Loire]"
7926,0.750,l,6.12,8.16,"Vins, bières, alcools",Vins rouges,Loire,NaN,"vin rouge aoc anjou les tréjeots, bouteille de...","Vin rouge AOC Anjou les tréjeots, bouteille de...",6.12,"8,16 € l",Système U,https://www.coursesu.com/dw/image/v2/BBQX_PRD/...,https://www.coursesu.com/p/vin-rouge-aoc-anjou...,3589604,"[Vins rouges, Vins, bières, alcools, Loire]"
7927,0.750,l,5.99,7.99,"Vins, bières, alcools",Vins rouges,Loire,NaN,vin rouge aop anjou domaine les grandes vignes...,Vin rouge AOP Anjou Domaine Les Grandes Vignes...,5.99,"7,99 € l",Système U,https://www.coursesu.com/dw/image/v2/BBQX_PRD/...,https://www.coursesu.com/p/vin-rouge-aop-anjou...,6131788,"[Vins rouges, Vins, bières, alcools, Loire]"
7928,0.750,l,3.96,5.28,"Vins, bières, alcools",Vins rouges,Loire,NaN,vin d'anjou aoc rouge domaine emile chupin cro...,Vin d'Anjou AOC rouge Domaine Emile Chupin Cro...,3.96,"5,28 € l",Système U,https://www.coursesu.com/dw/image/v2/BBQX_PRD/...,https://www.coursesu.com/p/vin-danjou-aoc-roug...,4226872,"[Vins rouges, Vins, bières, alcools, Loire]"
7929,0.750,l,2.85,3.80,"Vins, bières, alcools",Vins rouges,Rhône,NaN,vin rouge aop côtes du rhône cellier des dauph...,Vin rouge AOP Côtes du Rhône Cellier des Dauph...,2.85,"3,80 € l",Système U,https://www.coursesu.com/dw/image/v2/BBQX_PRD/...,https://www.coursesu.com/p/vin-rouge-aop-cotes...,2290950,"[Vins rouges, Vins, bières, alcools, Rhône]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8420,0.749,l,2.89,3.86,"Vins, bières, alcools",Autres digestifs,Sans alcool,MISTER COCKTAIL,cocktail sans alcool au citron vert et au kiwi...,Cocktail sans alcool au citron vert et au kiwi...,2.89,"3,86 € l",Système U,https://www.coursesu.com/dw/image/v2/BBQX_PRD/...,https://www.coursesu.com/p/cocktail-sans-alcoo...,86371,"[Autres digestifs, Vins, bières, alcools, Sans..."
8421,0.749,l,2.89,3.86,"Vins, bières, alcools",Autres digestifs,Sans alcool,MISTER COCKTAIL,cocktail sans alcool aux fruits de la passion ...,Cocktail sans alcool aux fruits de la passion ...,2.89,"3,86 € l",Système U,https://www.coursesu.com/dw/image/v2/BBQX_PRD/...,https://www.coursesu.com/p/cocktail-sans-alcoo...,87734,"[Autres digestifs, Vins, bières, alcools, Sans..."
8422,1.000,l,1.81,1.81,"Vins, bières, alcools",Autres digestifs,Sans alcool,NaN,"anisé sans alcool cazanova, 1l","Anisé sans alcool CAZANOVA, 1l",1.81,"1,81 € l",Système U,https://www.coursesu.com/dw/image/v2/BBQX_PRD/...,https://www.coursesu.com/p/anise-sans-alcool-c...,2066703,"[Autres digestifs, Vins, bières, alcools, Sans..."
8423,0.200,l,3.99,19.95,"Vins, bières, alcools",Autres digestifs,Digestifs,LA MAZIERE,"armagnac saint vivant , 40°, 20cl","Armagnac Saint Vivant LA MAZIERE, 40°, 20cl",3.99,"19,95 € l",Système U,https://www.coursesu.com/dw/image/v2/BBQX_PRD/...,https://www.coursesu.com/p/armagnac-saint-viva...,12459,"[Autres digestifs, Vins, bières, alcools, Dige..."
